In [1]:
!pip install -q -U datasets

In [2]:
!pip install -q git+https://github.com/huggingface/transformers.git@main accelerate

### Loading the model and tokenizer


In [3]:
import datasets

In [4]:
from transformers import AutoTokenizer
import transformers
import torch
from tqdm import notebook
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
model_id = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)

### Preparing the Pipeline


In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id, model_max_length=512, padding_side="left", add_eos_token=True
)

tokenizer.pad_token = tokenizer.eos_token

In [7]:
def tokenize(data):
    result = tokenizer(
        data["prompt"],
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [8]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
pipeline.tokenizer.pad_token_id = tokenizer.eos_token_id

### Generating Code


In [10]:
def llama_v2_prompt(messages: list[dict]):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    BOS, EOS = "<s>", "</s>"
    DEFAULT_SYSTEM_PROMPT = f"""Provide answers in CPP. Wrap around code in [CPP] and [/CPP]. Don't write any comments in generated code."""

    if messages[0]["role"] != "system":
        messages = [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT,
            }
        ] + messages

    messages = [
        {
            "role": messages[1]["role"],
            "content": B_SYS + messages[0]["content"] + E_SYS + messages[1]["content"],
        }
    ] + messages[2:]

    messages_list = [
        f"{BOS}{B_INST}{(prompt['content']).strip()}{E_INST}{(answer['content']).strip()}{EOS}"
        for prompt, answer in zip(messages[::2], messages[1::2])
    ]
    messages_list.append(f"{BOS}{B_INST}{(messages[-1]['content']).strip()}{E_INST}\n")

    return "".join(messages_list)

In [11]:
def generate_code(dataset):
    responses = []
    batch_count = 0
    batch_size = 2
    print(f"Total batches = {len(dataset)//batch_size}")
    for out in pipeline(
        KeyDataset(dataset, "prompt"),
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=512,
        batch_size=batch_size,
    ):
        batch_count += 1
        for o in out:
            responses.append(o["generated_text"])
        print("Processed batch[%d]" % batch_count, end="\r")
    return responses

In [12]:
def end_overlap(a, b):
    for i in range(0, len(a)):
        if b.startswith(a[i:]):
            return i
    return len(a)


def remove_prefix(s, prefix):
    prefix_end_idx = s.index(prefix) + len(prefix)
    return s[prefix_end_idx:]


def extract_code(prompt, response, code_context):
    B_CPP, E_CPP = "[CPP]", "[/CPP]"
    trimmed_response = remove_prefix(response, prompt)
    if trimmed_response.startswith(B_CPP):
        trimmed_response = remove_prefix(trimmed_response, B_CPP)
    try:
        suffix_idx = trimmed_response.index(E_CPP)
        trimmed_response = trimmed_response[:suffix_idx]
    except ValueError as ve:
        pass
    trimmed_response = trimmed_response.strip()
    code_context = code_context.strip()
    overlap_idx = end_overlap(code_context, trimmed_response)
    cleaned_response = code_context[:overlap_idx] + trimmed_response
    return cleaned_response


def save_file(path, code):
    with open(path, "w") as file:
        file.write(code)


def run_code_generation(dataset, save_path, n_passes=5):
    responses = generate_code(dataset)
    return responses

In [13]:
CODE_GENERATION_PATH = "./generations/cpp"

In [14]:
train_df = pd.read_csv("/kaggle/input/cpp-train-processed/train_processed.csv", index_col=0)
train_df.head()

,text,code,question,code_cleaned,functions,code_context
id,,,,,,
0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> \n using namespace st...,"['int maxPresum ( vector < int > a , vector < ...",#include <bits/stdc++.h> \n using namespace st...
1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...,#include <bits/stdc++.h> \n using namespace st...,"['bool sumOfTwoCubes ( int n ) {', 'int main (...",#include <bits/stdc++.h> \n using namespace st...
3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> \n using namespace st...,"['long findNthNumber ( long long N ) {', 'int ...",#include <bits/stdc++.h> \n using namespace st...
4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> \n using namespace st...,"['int check ( int A , int B ) {', 'int main ( ...",#include <bits/stdc++.h> \n using namespace st...
5,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> \n using namespace st...,"['void sameProductQuadruples ( int nums [ ] , ...",#include <bits/stdc++.h> \n using namespace st...


In [15]:
def build_prompt(question, code_context):
    prompt = f"# {question}\n{code_context}\n"
    instruction = {
        "role": "user",
        "content": prompt,
    }
    llama_prompt = llama_v2_prompt([instruction])
    return llama_prompt

In [16]:
def build_question_context(text, context):
    q_strs = text.split("|")
    q_strs = q_strs[1].split(";")[1:]
    return context

In [17]:
train_df_300 = train_df[0:300].copy()
train_df_300["context"] = train_df_300.apply(
    lambda row: build_question_context(row.text, row.code_context), axis=1
)
train_df_300["llama_prompt"] = train_df_300.apply(
    lambda row: build_prompt(row.question, row.code_context), axis=1
)
train_df_300.head()

,text,code,question,code_cleaned,functions,code_context,context,llama_prompt
id,,,,,,,,
0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> \n using namespace st...,"['int maxPresum ( vector < int > a , vector < ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...,#include <bits/stdc++.h> \n using namespace st...,"['bool sumOfTwoCubes ( int n ) {', 'int main (...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> \n using namespace st...,"['long findNthNumber ( long long N ) {', 'int ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> \n using namespace st...,"['int check ( int A , int B ) {', 'int main ( ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
5,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> \n using namespace st...,"['void sameProductQuadruples ( int nums [ ] , ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...


In [18]:
prompts = train_df_300["llama_prompt"].values.tolist()
prompts_dic = {"prompt": prompts}
prompts_dataset = Dataset.from_dict(prompts_dic)

In [19]:
prompts_dataset[0]

{'prompt': "<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap around code in [CPP] and [/CPP]. Don't write any comments in generated code.\n<</SYS>>\n\n# Maximum Prefix Sum possible by merging two given arrays.\n#include <bits/stdc++.h> \n using namespace std ;\n int maxPresum ( vector < int > a , vector < int > b ) {[/INST]\n"}

In [20]:
responses = run_code_generation(prompts_dataset, CODE_GENERATION_PATH, n_passes=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Total batches = 150


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [21]:
len(responses)

300

In [22]:
train_df_300["response"] = responses

train_df_300.head()

,text,code,question,code_cleaned,functions,code_context,context,llama_prompt,response
id,,,,,,,,,
0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> \n using namespace st...,"['int maxPresum ( vector < int > a , vector < ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...,#include <bits/stdc++.h> \n using namespace st...,"['bool sumOfTwoCubes ( int n ) {', 'int main (...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> \n using namespace st...,"['long findNthNumber ( long long N ) {', 'int ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> \n using namespace st...,"['int check ( int A , int B ) {', 'int main ( ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
5,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> \n using namespace st...,"['void sameProductQuadruples ( int nums [ ] , ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...


In [23]:
train_df_300.to_csv("./train_response.csv")